# Learning Unit 5 - Overfitting - Example

In [1]:
cd ..

/Users/pedro/PycharmProjects/workshop_viz/lisbon.ai-workshop/units


In [2]:
from ipywidgets import interact

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from bokeh.plotting import figure, output_notebook
output_notebook()

Loading BokehJS ...

In [4]:
from utils import load_data, visualizations

In [5]:
data = load_data.get_ying_yang(200)

In [6]:
visualizations.plot_data(model=KNeighborsClassifier(1), 
                     data=data, 
                     target='c', 
                     feature1='a', 
                     feature2='b', 
                     out_of_sample=False, 
                     probabilities=False)

In [7]:
visualizations.plot_data(model=KNeighborsClassifier(1), 
                     data=data, 
                     target='c', 
                     feature1='a', 
                     feature2='b', 
                     out_of_sample=True, 
                     probabilities=False)

In [8]:
def s(x):
    visualizations.plot_data(model=KNeighborsClassifier(n_neighbors=x), 
                     data=data, 
                     target='c', 
                     feature1='a', 
                     feature2='b', 
                     out_of_sample=True, 
                     probabilities=False)
    
interact(s, x=(1, 12))

interactive(children=(IntSlider(value=6, description='x', max=12, min=1), Output()), _dom_classes=('widget-interact',))

<function __main__.s>

In [9]:
def s(x, y):
    visualizations.plot_data(model=RandomForestClassifier(max_depth=x, n_estimators=y), 
                         data=data, 
                         target='c', 
                         feature1='a', 
                         feature2='b', 
                         out_of_sample=True, 
                         probabilities=False)
    
interact(s, x=(1, 15), y=(10, 100, 10))

interactive(children=(IntSlider(value=8, description='x', max=15, min=1), IntSlider(value=50, description='y', min=10, step=10), Output()), _dom_classes=('widget-interact',))

<function __main__.s>